In [ ]:
import pandas as pd
import numpy as np

from datetime import timedelta, datetime

import zucaml as ml

import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_columns', None)

In [ ]:
# todo: use from utils
def energy_to_mag(energy):

    energy_clipped = np.clip(energy, a_min = 1, a_max = None)
    energy_logged = np.log10(energy_clipped)

    mag = (energy_logged - 5.24) / 1.44

    mag = np.clip(mag, a_min = 0, a_max = None)

    return mag

def mag_to_energy(mag):

    energy = np.power(10, 1.44 * mag + 5.24)

    return energy

#### silver

In [ ]:
df_silver = pd.read_csv('data/silver.csv')

for feat in ['latitude', 'longitude', 'depth', 'mag']:
    df_silver[feat] = pd.to_numeric(df_silver[feat], downcast = 'float')
    
df_silver['date'] = pd.to_datetime(df_silver['date'], format = '%Y-%m-%d', exact = True)

df_silver = df_silver.dropna()

df_silver = df_silver[df_silver['mag'] > 0].reset_index(drop=True)

assert(df_silver['id'].nunique() == len(df_silver))

ml.print_memory(df_silver)
df_silver[:5]

#### Set variables

In [ ]:
# DataFrames
main_id = 'zone_frame'
time_ref = 'date'
time_frequency = '1D'
target_raw = 'energy'
early_warning_number = 1
range_warning_number = 1
pad_df = True

# pad before filters

# Problem
degrees_latitude_grid = 10
km_depth_grid = 100

# Filter
time_cut = datetime(1973, 1, 1)

# America
min_latitude = -50
max_latitude = 50
min_longitude = -135
max_longitude = -40

# # Asia
# min_latitude = -30
# max_latitude = 70
# min_longitude = 70
# max_longitude = 165

fe_1st_bound = 2
fe_granularity_1st = 30
fe_2nd_bound = 10
fe_granularity_2nd = 30 * 6
fe_mean_window = 30

fe_intervals = [i for i in range(0, (360 * fe_1st_bound) + fe_granularity_1st, fe_granularity_1st)]
print(f'first interval: {len(fe_intervals)}')
fe_1st_max = fe_intervals[len(fe_intervals) - 1]
fe_intervals += [i for i in range(fe_1st_max + fe_granularity_2nd, 360 * fe_2nd_bound + fe_granularity_2nd, fe_granularity_2nd)]

len(fe_intervals), fe_intervals[len(fe_intervals) - 1] / 360

In [ ]:
fe_intervals

#### filter regions

In [ ]:
df_silver['keep'] = (df_silver['latitude'] >= min_latitude)
df_silver['keep'] = df_silver['keep'] & (df_silver['latitude'] <= max_latitude) 
df_silver['keep'] = df_silver['keep'] & (df_silver['longitude'] >= min_longitude)
df_silver['keep'] = df_silver['keep'] & (df_silver['longitude'] <= max_longitude)

before_records = len(df_silver)

df_filtered = df_silver.loc[df_silver['keep']].copy().reset_index()

after_records = len(df_filtered)

df_filtered = df_filtered.drop([
    'keep',
    'index',
], axis = 1)

print(f'Decrease of events:\t\t{after_records / before_records - 1.0:.0%}\n')
print(f'Final number of events:\t{after_records:,d}\n')

ml.print_memory(df_filtered)
df_filtered[:5]

In [ ]:
latlon_dist = df_filtered.copy()

latlon_dist['longitude'] = latlon_dist['longitude'].round()
latlon_dist['latitude'] = latlon_dist['latitude'].round()

In [ ]:
lon_dist = latlon_dist.groupby('longitude').agg({'id': 'count'})['id']
plt.plot(lon_dist.index, lon_dist.values)

In [ ]:
lon_dist = latlon_dist.groupby('latitude').agg({'id': 'count'})['id']
plt.plot(lon_dist.index, lon_dist.values)

#### filter time

In [ ]:
plt.figure(figsize = (20, 10))

title_font_size = 18
axis_font_size = 14

plt.subplot(221).hist(df_filtered['date'], bins = 500)
plt.axvline(time_cut, color = 'green')
plt.title('All quakes', fontsize = title_font_size)
# plt.xlabel('Date', fontsize = 16)
# https://stackoverflow.com/questions/25973581/how-do-i-format-axis-number-format-to-thousands-with-a-comma-in-matplotlib
plt.gca().set_yticklabels(['{:,.0f}'.format(x) for x in plt.gca().get_yticks()], fontsize = axis_font_size)
# plt.gca().set_xticklabels([x for x in plt.gca().get_xticks()], fontsize = 14)

plt.subplot(222).hist(df_filtered[df_filtered['mag'] >= 2]['date'], bins = 500)
plt.axvline(time_cut, color = 'green')
plt.title('Magnitude >= ' + str(float(2)), fontsize = title_font_size)
y_values = plt.gca().get_yticks()
plt.gca().set_yticklabels(['{:,.0f}'.format(x) for x in y_values], fontsize = axis_font_size)

plt.subplot(223).hist(df_filtered[df_filtered['mag'] >= 5]['date'], bins = 500)
plt.axvline(time_cut, color = 'green')
plt.title('Magnitude >= ' + str(5), fontsize = title_font_size)
y_values = plt.gca().get_yticks()
plt.gca().set_yticklabels(['{:,.0f}'.format(x) for x in y_values], fontsize = axis_font_size)

plt.subplot(224).hist(df_filtered[df_filtered['mag'] >= 7]['date'], bins = 500)
plt.axvline(time_cut, color = 'green')
plt.title('Magnitude >= ' + str(float(7)), fontsize = title_font_size)
y_values = plt.gca().get_yticks()
plt.gca().set_yticklabels(['{:,.0f}'.format(x) for x in y_values], fontsize = axis_font_size)

In [ ]:
df_filtered = df_filtered[df_filtered['date'] > time_cut]

min_date = df_filtered['date'].min()
max_date = df_filtered['date'].max()

print(f'Min date:\t{min_date}\n')
print(f'Max date:\t{max_date}\n')

df_filtered = df_filtered.reset_index().drop([
    'index',
], axis = 1)

ml.print_memory(df_filtered)
df_filtered[:5]

#### energy

In [ ]:
df_energy = df_filtered.copy()

# TODO: use function
df_energy['energy'] = 5.24
df_energy['energy'] += 1.44 * df_energy['mag']
df_energy['energy'] = np.power(10, df_energy['energy'])

ml.print_memory(df_energy)
df_energy[:5]

#### grid

In [ ]:
def create_grid(dx, dy, dz):
    df = df_energy.copy()

    grid_values = [
        ('y', 'latitude', dy),
        ('x', 'longitude', dx),
        ('z', 'depth', dz)
    ]

    for new_feature, old_feature, increment in grid_values:

        old_feature_min = int(round(df[old_feature].min()))

        df[new_feature] = df[old_feature] - old_feature_min
        df[new_feature] = df[new_feature] / increment
        df[new_feature] = df[new_feature].round().astype(int)
        df[new_feature] = df[new_feature] * increment
        df[new_feature] = df[new_feature] + old_feature_min

    assert(sum(df['z'] < 0) == 0)

    df['zone_frame'] = df['x'].astype(str) + '|' + df['y'].astype(str) + '|' + df['z'].astype(str)

    min_x = df['x'].min()
    max_x = df['x'].max()
    min_y = df['y'].min()
    max_y = df['y'].max()
    min_z = df['z'].min()
    max_z = df['z'].max()

    range_x = range(min_x, max_x + dx, dx)
    range_y = range(min_y, max_y + dy, dy)
    range_z = range(min_z, max_z + dz, dz)

    all_zones = [str(x) + '|' + str(y) + '|' + str(z) for x in range_x for y in range_y for z in range_z]

    used_x = df['x'].nunique()
    used_y = df['y'].nunique()
    used_z = df['z'].nunique()
    used_time = df['date'].nunique()

    print(f'Unique x:\t\t{used_x:,d}\n')
    print(f'Unique y:\t\t{used_y:,d}\n')
    print(f'Unique z:\t\t{used_z:,d}\n')
    print(f'Unique time:\t\t{used_time:,d}\n')
    print(f'All zones:\t\t{len(all_zones):,d}\n')

    df = df.drop([
        'longitude',
        'latitude',
        'depth',
        'hour',
    ], axis = 1)

    return df, all_zones

x_degre_km = 94.2
y_degre_km = 111.2

resolution_y = degrees_latitude_grid
resolution_x = int(round(resolution_y * y_degre_km / x_degre_km))
resolution_z = km_depth_grid

df_grid, all_zone_frames = create_grid(resolution_x, resolution_y, resolution_z)

ml.print_memory(df_grid)
df_grid[:5]

In [ ]:
resolution_x, resolution_y, resolution_z,

#### all sectors, all timeframes

In [ ]:
###################################################
# used for convolutional networks
# adding min to all timeframes have same number of zones 
# in theroy the time filter should be done after reindexing
###################################################

if pad_df:

    zero_fill = ['mag', 'event', 'energy']
    other_fill = {'id': 'non_existant'}

    df_grid = ml.pad(df_grid, 'zone_frame', 'date', all_zone_frames, 'min', zero_fill, other_fill)
    df_grid = ml.pad(df_grid, 'zone_frame', 'date', all_zone_frames, 'max', zero_fill, other_fill)

    df_grid['x'] = df_grid['zone_frame'].str.split('|').str[0].astype(int)
    df_grid['y'] = df_grid['zone_frame'].str.split('|').str[1].astype(int)
    df_grid['z'] = df_grid['zone_frame'].str.split('|').str[2].astype(int)

    assert(df_grid.isna().sum().sum() == 0)

    ml.print_memory(df_grid)
    df_grid[:5]

In [ ]:
%%time

# reindex
df_full = ml.reindex_by_minmax(
    df = df_grid.drop(['mag', 'x', 'y', 'z'], axis = 1),
    item = main_id,
    time_ref = time_ref,
    time_freq = time_frequency,
    forwardfill_features = [],
    backfill_features = [],
    zerofill_features = ['energy'],
)

assert(df_full.isna().sum().sum() == 0)

df_full['x'] = df_full['zone_frame'].str.split('|').str[0].astype(int)
df_full['y'] = df_full['zone_frame'].str.split('|').str[1].astype(int)
df_full['z'] = df_full['zone_frame'].str.split('|').str[2].astype(int)

df_full = df_full.drop(['id'], axis=1)

ml.print_memory(df_full)
df_full[:5]

#### Set target

In [ ]:
%%time

df_gold = ml.set_target(
    df = df_full,
    item = main_id,
    time_ref = time_ref,
    target = target_raw,
    early_warning = early_warning_number,
    range_warning = range_warning_number,
    drop_na_target = True,
)

ml.print_memory(df_gold)
check = df_gold[df_gold['energy'] > 0].index[3]
df_gold[check - 5:check + 5]

In [ ]:
np.log(df_gold[df_gold['target'] > 0]['target']).hist()

In [ ]:
df_gold[df_gold['target'] > 0].groupby(time_ref).agg({main_id: 'count'})[main_id].max()

#### Feature engineering

In [ ]:
df_gold = ml.create_reset(
    df = df_gold,
    item = main_id,
    time_ref = time_ref,
    order = None
)

##### F.E.

In [ ]:
fe_features = []

features_before = [feature for feature in df_gold]

df_gold = ml.ts_feature(
    df = df_gold,
    feature_base = 'energy',
    func = 'rolling.mean',
    func_val = fe_mean_window,
    label = None,
)

base_feature = [feature for feature in df_gold if feature not in features_before][0]

for window_fe in fe_intervals:
    
    features_before = [feature for feature in df_gold]
    
    df_gold = ml.ts_feature(
        df = df_gold,
        feature_base = base_feature,
        func = 'shift',
        func_val = window_fe,
        label = None,
    )

    new_feat = [feature for feature in df_gold if feature not in features_before][0]

    print(f'New feature added: {new_feat}')

    df_gold[new_feat] = df_gold[new_feat].fillna(0)

    df_gold[new_feat] = energy_to_mag(df_gold[new_feat])

    df_gold[new_feat] = pd.to_numeric(df_gold[new_feat], downcast = 'float')

    fe_features.append(new_feat)

ml.print_memory(df_gold)
df_gold[:5]

In [ ]:
def get_years_span(df):
    
    max_year = df['date'].dt.year.max()
    min_year = df['date'].dt.year.min()
    
    print(f'Span: {max_year - min_year:,d} years - Max: {max_year:} - Min: {min_year:}')

In [ ]:
get_years_span(df_gold)

len_before = len(df_gold)

df_gold = df_gold.dropna()

print(f'Dropped {len(df_gold)/len_before - 1.0:.2%}')
get_years_span(df_gold)

ml.print_memory(df_gold)

##### Convert to mag to normalize the distribution

In [ ]:
df_gold['target'] = energy_to_mag(df_gold['target'])
if 'mid_term' in df_gold:
    df_gold['mid_term'] = energy_to_mag(df_gold['mid_term'])

##### Clean and order

In [ ]:
df_gold = (
    df_gold.drop(
        [feat for feat in ['reset', 'mag', base_feature] if feat in df_gold],
        axis = 1
    )
)

df_gold = df_gold.sort_values(['zone_frame', 'date']).reset_index().drop('index', axis = 1)

In [ ]:
df_gold.isna().sum().sort_values(ascending=False)

In [ ]:
check = df_gold[df_gold['energy'] > 0].index[3]

df_gold[check - 5:check + 8]

#### output

In [ ]:
unsigned_feats = ['z']
integer_feats = ['x', 'y']
float_feats = ['target']
for feat in df_gold:
    if feat.startswith('energy|rolling'):
        float_feats.append(feat)
    if feat.startswith('zone_frame_') and 'energy|rolling' in feat:
        float_feats.append(feat)
    if feat == 'event_mag':
        float_feats.append(feat)

In [ ]:
df_gold = ml.downcast_csv(df_gold, unsigned_feats, integer_feats, float_feats, True)

In [ ]:
df_gold['target'].hist()

In [ ]:
(df_gold[df_gold['target'] > 0]['target']).hist()

In [ ]:
ml.print_balances([df_gold], len(df_gold), 'target', ml.problems.MULTICLASS)

In [ ]:
df_temp = df_gold.loc[:, ['date', 'target']].copy()

df_temp['event'] = (df_temp['target'] > 0) * 1

df_temp = df_temp.groupby('date')['event'].sum().reset_index().sort_values('event')

no_events = (df_temp['event'] == 0).sum()

print(f'No events: {no_events / len(df_temp):.2%}')

In [ ]:
%%time

df_gold.to_csv('data/gold.csv', index=False)